# Safety-first Risk Measures

Developed based on the book Handbook of Finance, the 2 safety measures that were requested to be included in the financial dashboard were the Minimax (MM) and Value at Risk (VaR) measures. These measures are computed to help understand the value that an investor could lose out on if they were to invest in the stock. While these may be short sided measures, it could help an investor understand what to expect in the short term to take on more risk in the long term.

We first start by importing some packages that allow us to utilize the MongoDB Altas cloud database and access a financial packages that simplifies calculating the two well know safety measures. These pages are also installed when the requirements files is run.

In [ ]:
# pip install pyfolio
# pip install pymongo
# pip install "pymongo[srv]"

## Retrieving the Data
Joshua R has spun up a MongoDB Atlas instance and provided the team with credentials. Using the Yahoo data, I will be able to perform VAR and MM with the pyfolio package. I'll first log into the database to develop an understand of the contents.

In [64]:
import pyfolio as pf
import pymongo
import configparser
from pprint import pprint
from typing import Union

config = configparser.ConfigParser()
config.read('config.ini')
mongo_config = config['MONGO']
mongo_connection = "mongodb+srv://" + mongo_config['User'] + ":" + mongo_config['Password'] + "@" + mongo_config['Address'] + "/test"
myclient = pymongo.MongoClient(mongo_connection)


In [46]:
mydb = myclient["market_shopper"]
sub_col = mydb["sec_sub"]
num_col = mydb["sec_num"]
tag_col = mydb["sec_tag"]
pre_col = mydb["sec_pre"]
sec_col = mydb["sec"]
yahoo_col = mydb["yahoo"]

In [82]:
def retrieve_close(ticker:'str',start:Union[None,dt.datetime]=dt.date.today(),end:Union[None,dt.datetime]=dt.date.today()):
    t = dt.datetime(2000,1,1,12)
    detail = {'ticker':ticker}
    rows = yahoo_col.find_one(detail)['stock_price']
    close = [(row['Date'],row['Close']) for row in rows if row['Date'].date() >= start and row['Date'].date() <= end]
    return close
print(retrieve_close('SPY', dt.datetime(2020,1,1).date()))
# print(yahoo_col.find_one({'ticker':'SPY'})['stock_price'])

[(datetime.datetime(2020, 1, 2, 5, 0), 311.1169738769531), (datetime.datetime(2020, 1, 3, 5, 0), 308.7611389160156), (datetime.datetime(2020, 1, 6, 5, 0), 309.93914794921875), (datetime.datetime(2020, 1, 7, 5, 0), 309.06768798828125), (datetime.datetime(2020, 1, 8, 5, 0), 310.7148132324219), (datetime.datetime(2020, 1, 9, 5, 0), 312.8216552734375), (datetime.datetime(2020, 1, 10, 5, 0), 311.9215087890625), (datetime.datetime(2020, 1, 13, 5, 0), 314.06671142578125), (datetime.datetime(2020, 1, 14, 5, 0), 313.5877990722656), (datetime.datetime(2020, 1, 15, 5, 0), 314.2965087890625), (datetime.datetime(2020, 1, 16, 5, 0), 316.91094970703125), (datetime.datetime(2020, 1, 17, 5, 0), 317.8973083496094), (datetime.datetime(2020, 1, 21, 5, 0), 317.2747802734375), (datetime.datetime(2020, 1, 22, 5, 0), 317.3131103515625), (datetime.datetime(2020, 1, 23, 5, 0), 317.67706298828125), (datetime.datetime(2020, 1, 24, 5, 0), 314.8519287109375), (datetime.datetime(2020, 1, 27, 5, 0), 309.8050231933594

In [10]:
# from pandas_datareader import data as pdr
import yfinance as yf
import datetime as dt
import pandas as pd
# yf.pdr_override() # Use Pandas data reader to extract yf data faster for development purposes.

In [22]:
## Function for retrieving data in the interrium. Will replace with database function

def retrieve_data(ticker: str, period:str):
    data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = ticker,

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = period,

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = False,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )
    return data

def get_noncumulative_returns(df:pd.DataFrame):
    return df['Close'].pct_change()

def define_value_at_risk(portfolio_returns:pd.DataFrame,baseline_returns:pd.DataFrame):
    return pf.show_perf_stats(portfolio_returns,baseline_returns)


## VAR
We will compare the noncumulative results for a given ti

In [26]:
# TODO: Turn into unit test.
# This test emulates someone with a portfolio of just one stock
# The performance is measured against that of SPY, which is a 
# standard baseline for the US Market.
bac = retrieve_data('BAC', '1mo')
spy = retrieve_data('SPY', '1mo')

bac_returns = get_noncumulative_returns(bac)
spy_returns = get_noncumulative_returns(spy)

define_value_at_risk(bac_returns,spy_returns)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Start date,2022-09-15
End date,2022-10-14
Total months,1
,Backtest
Annual return,-62.2%
Cumulative returns,-8.1%
Annual volatility,38.3%
Sharpe ratio,-2.47
Calmar ratio,-4.39
Stability,0.45
Max drawdown,-14.2%


## MiniMax

Explain minimax calculation

## Visualizing with pyChart.js